<a href="https://colab.research.google.com/github/eichi7/internshipHachi-x/blob/main/02_Objectdetection20220810ingoooglecolab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt
import os 
import cv2



In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
df=pd.read_csv('/content/drive/MyDrive/1_Labeling/labelsdrive.csv')
df

,filepath,xmin,xmax,ymin,ymax
0,/content/drive/MyDrive/1_Labeling/images/N1.xml,1093,1396,645,727
1,/content/drive/MyDrive/1_Labeling/images/N100.xml,134,301,312,350
2,/content/drive/MyDrive/1_Labeling/images/N101.xml,31,139,128,161
3,/content/drive/MyDrive/1_Labeling/images/N102.xml,164,316,216,243
4,/content/drive/MyDrive/1_Labeling/images/N103.xml,813,1067,665,724
...,...,...,...,...,...
220,/content/drive/MyDrive/1_Labeling/images/N95.xml,23,408,173,391
221,/content/drive/MyDrive/1_Labeling/images/N96.xml,137,352,141,186
222,/content/drive/MyDrive/1_Labeling/images/N97.xml,175,290,228,255
223,/content/drive/MyDrive/1_Labeling/images/N98.xml,563,675,207,238


In [4]:
import xml.etree.ElementTree as xet #use for enter in the xml file

In [5]:
filename= df['filepath'][0]
filename

'/content/drive/MyDrive/1_Labeling/images/N1.xml'

In [6]:
def getFilename(filename):
    filename_image=xet.parse(filename).getroot().find('filename').text#get the path
    #filepath_image=os.path.join('./images',filename_image)
    #return filepath_image
    return os.path.join('/content/drive/MyDrive/1_Labeling/images',filename_image)

In [7]:
getFilename(filename)

'/content/drive/MyDrive/1_Labeling/images/N1.jpeg'

In [8]:
image_path=list(df['filepath'].apply(getFilename))

In [9]:
image_path

['/content/drive/MyDrive/1_Labeling/images/N1.jpeg',
 '/content/drive/MyDrive/1_Labeling/images/N100.jpeg',
 '/content/drive/MyDrive/1_Labeling/images/N101.jpeg',
 '/content/drive/MyDrive/1_Labeling/images/N102.jpeg',
 '/content/drive/MyDrive/1_Labeling/images/N103.jpeg',
 '/content/drive/MyDrive/1_Labeling/images/N104.jpeg',
 '/content/drive/MyDrive/1_Labeling/images/N105.jpeg',
 '/content/drive/MyDrive/1_Labeling/images/N106.jpeg',
 '/content/drive/MyDrive/1_Labeling/images/N108.jpeg',
 '/content/drive/MyDrive/1_Labeling/images/N107.jpeg',
 '/content/drive/MyDrive/1_Labeling/images/N109.jpeg',
 '/content/drive/MyDrive/1_Labeling/images/N11.jpeg',
 '/content/drive/MyDrive/1_Labeling/images/N110.jpeg',
 '/content/drive/MyDrive/1_Labeling/images/N111.jpeg',
 '/content/drive/MyDrive/1_Labeling/images/N112.jpeg',
 '/content/drive/MyDrive/1_Labeling/images/N113.jpeg',
 '/content/drive/MyDrive/1_Labeling/images/N114.jpeg',
 '/content/drive/MyDrive/1_Labeling/images/N116.jpeg',
 '/content/dr

In [10]:
file_path=image_path[0]
file_path

'/content/drive/MyDrive/1_Labeling/images/N1.jpeg'

In [11]:
img=cv2.imread(file_path)

#cv2.namedWindow('example',cv2.WINDOW_NORMAL)
#cv2.imshow('example',img)
#cv2.waitKey(0) #if 0 is passed in the argument it waits till any key is pressed
#cv2.destroyAllWindows()'''

In [ ]:
#207356174287

cv2.rectangle(img,(207,174),(356,287),(0,255,0),3)
cv2.namedWindow('example',cv2.WINDOW_NORMAL)
cv2.imshow('example',img)
cv2.waitKey(0) #if 0 is passed in the argument it waits till any key is pressed
cv2.destroyAllWindows()

In [12]:
### DATA PREPROCESSING ####

from sklearn.model_selection import train_test_split

In [13]:

import tensorflow
from tensorflow.keras.preprocessing.image import load_img,img_to_array


In [14]:
labels=df.iloc[:,1:].values # find the value in the dataframe

In [15]:
data=[]
output=[]
for ind in range(len(image_path)):
    image=image_path[ind]
    img_arr=cv2.imread(image)
    h,w,d=img_arr.shape
    #preprocessing
    load_image= load_img(image,target_size=(224,224))
    load_image_arr=img_to_array(load_image)
    norm_load_image_arr=load_image_arr/225.0 
    #normalization to labels 
    xmin,xmax,ymin,ymax = labels[ind]
    nxmin,nxmax = xmin/w,xmax/w
    nymin,nymax = ymin/h,ymax/h
    label_norm = (nxmin,nxmax,nymin,nymax) # normalized output
    # -------------- append
    data.append(norm_load_image_arr)
    output.append(label_norm)

In [16]:
x= np.array(data,dtype=np.float32)
y = np.array(output,dtype=np.float32)

In [17]:
x.shape,y.shape

((225, 224, 224, 3), (225, 4))

In [18]:
x_train,x_test,y_train,y_test = train_test_split(x,y,train_size=0.8,random_state=0)
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((180, 224, 224, 3), (45, 224, 224, 3), (180, 4), (45, 4))

# Deep learning Model

In [19]:
from tensorflow.keras.applications import MobileNetV2,InceptionV3,InceptionResNetV2
from tensorflow.keras.layers import Dense,Dropout,Flatten,Input
from tensorflow.keras.models import Model
import tensorflow as tf

In [20]:
inception_resnet=InceptionResNetV2(weights="imagenet",include_top=False,input_tensor=Input(shape=(224,224,3)))
inception_resnet.trainable=False
#-------------------------------------------------------------------------Neural Network
headmodel=inception_resnet.output
headmodel=Flatten()(headmodel)
headmodel=Dense(500,activation="relu")(headmodel)#layer
headmodel=Dense(400,activation="relu")(headmodel)
headmodel=Dense(250,activation="relu")(headmodel)
headmodel=Dense(4,activation="sigmoid")(headmodel)
#-----------model
model=Model(inputs=inception_resnet.input,outputs=headmodel)

219070464/219055592 [==============================] - 1s 0us/step


# Compile model

In [21]:

from tensorflow.keras.optimizers import Adam
model.compile(loss='mse',optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4))
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 111, 111, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 111, 111, 32  96         ['conv2d[0][0]']                 
 alization)                     )                                                             

# model training

In [22]:
from tensorflow.keras.callbacks import TensorBoard

In [25]:
tfb=TensorBoard('/content/drive/MyDrive/1_Labeling/object_detectiondrive')

In [26]:
history= model.fit(x=x_train,y=y_train,batch_size=10,epochs=100,validation_data=(x_test,y_test),callbacks=[tfb])

Epoch 1/100
18/18 [==============================] - 67s 4s/step - loss: 0.0485 - val_loss: 0.0176
Epoch 2/100
18/18 [==============================] - 66s 4s/step - loss: 0.0137 - val_loss: 0.0124
Epoch 3/100
18/18 [==============================] - 64s 4s/step - loss: 0.0066 - val_loss: 0.0082
Epoch 4/100
18/18 [==============================] - 65s 4s/step - loss: 0.0051 - val_loss: 0.0069
Epoch 5/100
18/18 [==============================] - 56s 3s/step - loss: 0.0037 - val_loss: 0.0074
Epoch 6/100
18/18 [==============================] - 55s 3s/step - loss: 0.0027 - val_loss: 0.0061
Epoch 7/100
18/18 [==============================] - 63s 4s/step - loss: 0.0026 - val_loss: 0.0073
Epoch 8/100
18/18 [==============================] - 53s 3s/step - loss: 0.0020 - val_loss: 0.0063
Epoch 9/100
18/18 [==============================] - 54s 3s/step - loss: 0.0021 - val_loss: 0.0071
Epoch 10/100
18/18 [==============================] - 63s 4s/step - loss: 0.0016 - val_loss: 0.0064
Epoch 11/

In [27]:
history= model.fit(x=x_train,y=y_train,batch_size=10,epochs=200,validation_data=(x_test,y_test),callbacks=[tfb],initial_epoch=101)

Epoch 102/200
18/18 [==============================] - 54s 3s/step - loss: 2.8396e-04 - val_loss: 0.0054
Epoch 103/200
18/18 [==============================] - 55s 3s/step - loss: 2.8797e-04 - val_loss: 0.0056
Epoch 104/200
18/18 [==============================] - 56s 3s/step - loss: 3.5365e-04 - val_loss: 0.0052
Epoch 105/200
18/18 [==============================] - 63s 4s/step - loss: 3.8938e-04 - val_loss: 0.0063
Epoch 106/200
18/18 [==============================] - 66s 4s/step - loss: 4.1843e-04 - val_loss: 0.0055
Epoch 107/200
18/18 [==============================] - 54s 3s/step - loss: 4.4450e-04 - val_loss: 0.0070
Epoch 108/200
18/18 [==============================] - 66s 4s/step - loss: 5.7973e-04 - val_loss: 0.0063
Epoch 109/200
18/18 [==============================] - 64s 4s/step - loss: 4.0380e-04 - val_loss: 0.0055
Epoch 110/200
18/18 [==============================] - 56s 3s/step - loss: 2.6332e-04 - val_loss: 0.0056
Epoch 111/200
18/18 [==============================] - 

In [28]:
model.save('/content/drive/MyDrive/1_Labeling/models/object_detection.h6')

In [29]:
!tensorboard --logdir="/content/drive/MyDrive/1_Labeling/object_detectiondrive"

2022-08-11 07:25:11.024188: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected

NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.8.0 at http://localhost:6006/ (Press CTRL+C to quit)
^C
